# Codage super dense

## Description du protocole


_"En quittant Athènes, Thésée avait promis à son père Egée de hisser une voile blanche à son retour en cas de victoire sur le Minotaure, et de laisser une voile noire hissée en cas d'échec.
Chaque matin, Egée se porte à l'horizon, espérant apercevoir une voile blanche."_

Transposons cette histoire mythologique dans notre monde moderne quantique, en changeant un peu les noms des protagonistes : Alice souhaite communiquer une couleur à Bob (Noir, Rouge, Bleu, ou Blanc) en utilisant la mécanique quantique.

Dans cet exercice, il s'agit donc de transmettre une information classique composée de deux bits 0.0 (Noir) ou 0.1 (Rouge) ou 1.0 (Bleu) ou 1.1 (Blanc) en transmettant un seul qubit via un protocole appelé le **codage super dense**.


Voici la description de l'algorithme :

- Une personne quelconque, disons un ami de Alice, Charlie, commence par intriquer deux qubits entre eux : pour cela, il prépare l'état de Bell $|\phi^+\rangle = \frac{1}{\sqrt{2}}|0.0\rangle + \frac{1}{\sqrt{2}}|1.1\rangle $, qui correspond à une intrication maximale. 

- En notant l'état de Bell de la forme : $|\phi^+\rangle = \frac{1}{\sqrt{2}}|0_A.0_B\rangle + \frac{1}{\sqrt{2}}|1_A.1_B\rangle $, Charlie envoie :
    - un premier qubit à Alice ($|\psi_A\rangle = \frac{1}{\sqrt{2}}|0_A\rangle + \frac{1}{\sqrt{2}}|1_A\rangle $)
    - un deuxième qubit à Bob ($|\psi_B\rangle = \frac{1}{\sqrt{2}}|0_B\rangle + \frac{1}{\sqrt{2}}|1_B\rangle $)  

Attention ! Ces deux qubits sont **intriqués**, c'est-à-dire qu'ils continuent de partager des données communes même séparés d'une très grande distance. C'est sur cette particularité que va jouer le codage super dense : si $|\psi_A\rangle$ donne une valeur lors de sa mesure, alors $|\psi_B\rangle$ vaudra aussi cette valeur, et inversement.


- Maintenant que Alice a en main son qubit $|\psi_A\rangle$, il s'agit de l'utiliser pour faire passer son message à Bob. Admettons les règles suivantes (vous allez comprendre par la suite pourquoi) :
    - Si elle veut transmettre Noir / 0.0, disons qu'elle applique l'Identité _I_ sur son qubit (il ne se passe rien)
    - Si elle veur transmettre l'information Rouge / 0.1, elle applique la porte _X_
    - Si elle veur transmettre l'information Bleu / 1.0, elle applique la porte _Z_
    - Si elle veur transmettre l'information Rouge / 1.1, elle applique la porte _X_, suivie de la porte _Z_
    
- Très bien, Alice a transformé son qubit comme elle voulait, elle l'envoie donc à Bob. Ce qubit sera noté $|\psi_A'\rangle$.


- Bob reçoit son qubit $|\psi_A'\rangle$ de la part d'Alice. Mais !! N'oublions pas qu'il a en main son qubit $|\psi_B\rangle = \frac{1}{\sqrt{2}}|0_B\rangle + \frac{1}{\sqrt{2}}|1_B\rangle $ ! Et comme nous l'avons dit plus haut, les deux qubits $|\psi_A\rangle$ et $|\psi_B\rangle$ sont intriqués, c'est-à-dire que la mesure de l'un donne la mesure de l'autre.

Ainsi (cette résolution peut être prouvée), si Bob applique exactement la porte $CNOT$ sur le 2-qubit puis la porte $H$ sur le qubit indexé par A, et retrouve le message d'Alice !


    


## Mise en pratique

In [33]:
import qiskit as q
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from qiskit import BasicAer

In [49]:
simulator = q.BasicAer.get_backend('qasm_simulator')
circuit = q.QuantumCircuit(2,2)

### 1) Préparation de l'état de Bell

La première étape du codage super dense consiste pour Charlie, l'ami d'Alice et Bob, à la préparation de l'état dit "de Bell", qui consiste en l'application d'une porte $H$ suivie d'une porte $X$.

In [50]:
circuit.h(0)
circuit.cx(0,1)

print(circuit.draw(output = "text"))

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 2/══════════
               


Choisissons un message bidon pour Alice, composé de deux bits :

In [57]:
message_alice = "11"

### 2) Codage d'Alice

Alice procède ensuite au codage en appliquant différentes portes quantiques selon son message.

In [ ]:
def transformation_alice(message):
    if message == '00':
        circuit.iden(0)
    elif message == '01':
        circuit.z(0)
    elif message == '10':
        circuit.x(0)
    elif message == '11':
        circuit.x(0)
        circuit.z(0)


In [58]:
transformation_alice(message_alice)

### 3) Decodage de Bob

Bob procède ensuite au décodage, en appliquant l'inverse du processus appliqué par Charlie, soit la porte $X$ suivie de la porte $H$.

In [54]:
# Décodage par Bob
circuit.cx(0,1)
circuit.h(0)

In [61]:
circuit.measure([0,1], [0,1])
print(circuit.draw(output='text'))

     ┌───┐     ┌───┐┌───┐     ┌───┐┌─┐┌───┐┌───┐┌─┐
q_0: ┤ H ├──■──┤ X ├┤ Z ├──■──┤ H ├┤M├┤ X ├┤ Z ├┤M├
     └───┘┌─┴─┐└───┘└───┘┌─┴─┐└┬─┬┘└╥┘└┬─┬┘└───┘└╥┘
q_1: ─────┤ X ├──────────┤ X ├─┤M├──╫──┤M├───────╫─
          └───┘          └───┘ └╥┘  ║  └╥┘       ║ 
c: 2/═══════════════════════════╩═══╩═══╩════════╩═
                                1   0   1        0 


Executons l'ensemble du protocole :

In [62]:
# Execution
job = q.execute(circuit, simulator, shots=1000)
result = job.result()
counts = result.get_counts(circuit)
print(counts)

{'10': 1000}


Nous voyons ici que Bob optient le message d'Alice dans 100% des cas !